In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from time import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn  as sns
from sklearn.metrics import mean_squared_error

In [3]:
from utils import (
    MyPlotGrid,
    print_distribution,
    my_distributions_view,
    split_dataset,
    data_preparation_pipe,
    global_pipeline_train,
    update_results,
)

In [4]:
!pwd

/media/disk/home/pau/Projectes/typeform_test/notebooks


In [5]:
results = {
    'name': [],
    'prefix' : [],
    'model_class' : [],
    'trained_score' : [],
    'scorer_func' : [],
    'params' : [],
    'test_score': [],
}

In [6]:
df = pd.read_csv('../data/completion_rate.csv')

In [7]:
df.head()

,form_id,views,submissions,feat_01,feat_02,feat_03,feat_04,feat_05,feat_06,feat_07,...,feat_38,feat_39,feat_40,feat_41,feat_42,feat_43,feat_44,feat_45,feat_46,feat_47
0,1113027,33,27,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0,2.0
1,1115313,147,111,0.0,2.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1115907,528,136,0.0,1.0,0.0,0.0,1.0,0.0,6.0,...,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,30.0
3,1116299,55,21,0.0,2.0,0.0,0.0,0.0,1.0,2.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,7.0
4,1120373,62,54,0.0,0.0,0.0,0.0,1.0,0.0,4.0,...,0.0,0.0,0.0,0.0,1.0,1.0,1.0,11.0,7.0,21.0


In [8]:
df.columns

Index(['form_id', 'views', 'submissions', 'feat_01', 'feat_02', 'feat_03',
       'feat_04', 'feat_05', 'feat_06', 'feat_07', 'feat_08', 'feat_09',
       'feat_10', 'feat_11', 'feat_12', 'feat_13', 'feat_14', 'feat_15',
       'feat_16', 'feat_17', 'feat_18', 'feat_19', 'feat_20', 'feat_21',
       'feat_22', 'feat_23', 'feat_24', 'feat_25', 'feat_26', 'feat_27',
       'feat_28', 'feat_29', 'feat_30', 'feat_31', 'feat_32', 'feat_33',
       'feat_34', 'feat_35', 'feat_36', 'feat_37', 'feat_38', 'feat_39',
       'feat_40', 'feat_41', 'feat_42', 'feat_43', 'feat_44', 'feat_45',
       'feat_46', 'feat_47'],
      dtype='object')

In [53]:
df.describe()

,form_id,views,submissions,feat_01,feat_02,feat_03,feat_04,feat_05,feat_06,feat_07,...,feat_38,feat_39,feat_40,feat_41,feat_42,feat_43,feat_44,feat_45,feat_46,feat_47
count,1.031284e+06,1.031284e+06,1.031284e+06,1.031284e+06,1.031284e+06,1.031284e+06,1.031284e+06,1.031284e+06,1.031284e+06,1.031284e+06,...,1.031284e+06,1.031284e+06,1.031284e+06,1.031284e+06,1.031284e+06,1.031284e+06,1.031284e+06,1.031284e+06,1.031284e+06,1.031284e+06
mean,7.987444e+06,9.395541e+02,3.224132e+02,1.954515e-01,3.749122e-01,5.497322e-02,1.024936e-03,1.449494e-01,1.769047e-01,8.513950e-01,...,2.399572e-01,5.879176e-02,1.948299e-01,1.551163e-01,2.771681e-01,3.399791e-01,4.803430e-02,8.318746e-01,6.470032e-01,4.468738e+00
std,3.965056e+06,2.674871e+04,8.507656e+03,3.965481e-01,9.334260e-01,4.403415e-01,3.307117e-02,4.802633e-01,9.472932e-01,1.883366e+00,...,1.266072e+00,4.198431e-01,1.173311e+00,7.668222e-01,5.640830e-01,6.157581e-01,2.834470e-01,4.302039e+00,6.045358e+00,1.085162e+01
min,2.255130e+05,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,3.849671e+06,3.600000e+01,1.700000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,9.887946e+06,7.900000e+01,3.300000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
75%,1.149435e+07,2.260000e+02,9.000000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,5.000000e+00
max,1.350100e+07,1.226718e+07,6.754385e+06,1.000000e+00,5.500000e+01,7.800000e+01,5.000000e+00,8.700000e+01,9.600000e+01,1.240000e+02,...,1.050000e+02,4.600000e+01,1.000000e+02,1.350000e+02,8.700000e+01,8.700000e+01,8.700000e+01,5.900000e+02,2.065000e+03,1.145000e+03


In [54]:
# missing data?
print(len(df))
df.info()

1031284
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1031284 entries, 0 to 1031283
Data columns (total 50 columns):
 #   Column       Non-Null Count    Dtype  
---  ------       --------------    -----  
 0   form_id      1031284 non-null  int64  
 1   views        1031284 non-null  int64  
 2   submissions  1031284 non-null  int64  
 3   feat_01      1031284 non-null  float64
 4   feat_02      1031284 non-null  float64
 5   feat_03      1031284 non-null  float64
 6   feat_04      1031284 non-null  float64
 7   feat_05      1031284 non-null  float64
 8   feat_06      1031284 non-null  float64
 9   feat_07      1031284 non-null  float64
 10  feat_08      1031284 non-null  float64
 11  feat_09      1031284 non-null  float64
 12  feat_10      1031284 non-null  float64
 13  feat_11      1031284 non-null  float64
 14  feat_12      1031284 non-null  float64
 15  feat_13      1031284 non-null  float64
 16  feat_14      1031284 non-null  float64
 17  feat_15      1031284 non-null  float64

In [55]:
# since we are going to predict the ratio, let's compute it now and look for outliers in it
#df['completion_ratio'] = df['submissions']/df['views']

In [9]:
# we will remove the 'form_id', 'views', 'submissions', columns from the distribution visualization, since we are not interested on those

selected_columns = list(df.columns)
selected_columns.pop(selected_columns.index('form_id'))
selected_columns.pop(selected_columns.index('views'))
selected_columns.pop(selected_columns.index('submissions'))
print(selected_columns)

['feat_01', 'feat_02', 'feat_03', 'feat_04', 'feat_05', 'feat_06', 'feat_07', 'feat_08', 'feat_09', 'feat_10', 'feat_11', 'feat_12', 'feat_13', 'feat_14', 'feat_15', 'feat_16', 'feat_17', 'feat_18', 'feat_19', 'feat_20', 'feat_21', 'feat_22', 'feat_23', 'feat_24', 'feat_25', 'feat_26', 'feat_27', 'feat_28', 'feat_29', 'feat_30', 'feat_31', 'feat_32', 'feat_33', 'feat_34', 'feat_35', 'feat_36', 'feat_37', 'feat_38', 'feat_39', 'feat_40', 'feat_41', 'feat_42', 'feat_43', 'feat_44', 'feat_45', 'feat_46', 'feat_47']


* We observe that features 21 and 23 are highly correlated
* we observe that features 45 and 21 are highly correlated
* we observe that features 45 and 46 are highly correlated
* we could try to remove features 05, 07 and 33 also

## Data Preparation


* Sampling the dataset to work faster in the prototype
* Splitting the dataset into train, validation, test
* Generating a scikit-learn pipeline that includes neede processing of the data
    * Generating the target column named "completion_ration"
    * remove unwanted columns (ItemSelector)
    * include the option to remove the highly correlated variables (ItemSelector)

## Model training prototype
### Baseline linear regression

In [10]:
df = df.sample(frac=0.4)
print(len(df))
X_train_val, X_train, X_validation, X_test = split_dataset(df)
print(len(X_train_val),len(X_train),len(X_validation),len(X_test))

412514
330011 264008 66003 82503


In [11]:
# look how to generate the target on the pipeline and then train
start = time()
result_line = global_pipeline_train(
    all_features=selected_columns,
    numerical_features=selected_columns,
    params_grid=[
        {
        },
    ],
    score='neg_root_mean_squared_error',
    score_cls = mean_squared_error,
    X_train_val=X_train_val,
    X_test=X_test,
    target_column='completion_ratio',
    training_job_name='LinearRegr',
    model_type='linearRegression',
)    
end = time()
print("Total time", end-start,"s")
update_results(results, result_line)
pd.DataFrame(results).round(5)


Total time 4.3924150466918945 s


,name,prefix,model_class,trained_score,scorer_func,params,test_score
0,LinearRegr,clf,LinearRegression(),-0.21619,mean_squared_error,{},0.21684


### Ridge regression

In [59]:
# look how to generate the target on the pipeline and then train
start = time()
result_line = global_pipeline_train(
    all_features=selected_columns,
    numerical_features=selected_columns,
    params_grid=[
        {
            'clf__alpha':[ 10,1,0.5,0.1,0.01,0.001],
        },
    ],
    score='neg_root_mean_squared_error',
    score_cls = mean_squared_error,
    X_train_val=X_train_val,
    X_test=X_test,
    target_column='completion_ratio',
    training_job_name='RidgeRegr',
    model_type='Ridge',
)    

end = time()
print("Total time", end-start,"s")
update_results(results, result_line)
pd.DataFrame(results).round(5)


Total time 8.495603799819946 s


,name,prefix,model_class,trained_score,scorer_func,params,test_score
0,LinearRegr,clf,LinearRegression(),-0.21631,mean_squared_error,{},0.21623
1,RidgeRegr,clf,Ridge(),-0.21631,mean_squared_error,{'clf__alpha': 0.01},0.21622


### Regression with Decision trees

In [60]:
# look how to generate the target on the pipeline and then train
start = time()
result_line = global_pipeline_train(
    all_features=selected_columns,
    numerical_features=selected_columns,
    params_grid=[
        {
            'clf__max_depth':[ 2,4,16],
        },
    ],
    score='neg_root_mean_squared_error',
    score_cls = mean_squared_error,
    X_train_val=X_train_val,
    X_test=X_test,
    target_column='completion_ratio',
    training_job_name='RandomForestRegr',
    model_type='RandomForestRegressor',
)    

end = time()
print("Total time", end-start,"s")
update_results(results, result_line)
pd.DataFrame(results).round(5)

Total time 962.0871036052704 s


,name,prefix,model_class,trained_score,scorer_func,params,test_score
0,LinearRegr,clf,LinearRegression(),-0.21631,mean_squared_error,{},0.21623
1,RidgeRegr,clf,Ridge(),-0.21631,mean_squared_error,{'clf__alpha': 0.01},0.21622
2,RandomForestRegr,clf,RandomForestRegressor(),-0.20925,mean_squared_error,{'clf__max_depth': 16},0.20890


### Using a bigger sample

In [61]:
def my_current_train(restart=False, suffix=''):
    global results
    if restart:
        results = {
            'name': [],
            'prefix' : [],
            'model_class' : [],
            'trained_score' : [],
            'scorer_func' : [],
            'params' : [],
            'test_score': [],
        }


    start = time()

    # Linear regression
    result_line = global_pipeline_train(
        all_features=selected_columns,
        numerical_features=selected_columns,
        params_grid=[
            {
            },
        ],
        score='neg_root_mean_squared_error',
        score_cls = mean_squared_error,
        X_train_val=X_train_val,
        X_test=X_test,
        target_column='completion_ratio',
        training_job_name='LinearRegr'+'_'+suffix,
        model_type='linearRegression',
    )    
    end = time()
    print("Total time", end-start,"s")
    update_results(results, result_line)
    pd.DataFrame(results).round(5)

    # Ridge regression
    result_line = global_pipeline_train(
        all_features=selected_columns,
        numerical_features=selected_columns,
        params_grid=[
            {
                'clf__alpha':[ 0.5,],
            },
        ],
        score='neg_root_mean_squared_error',
        score_cls = mean_squared_error,
        X_train_val=X_train_val,
        X_test=X_test,
        target_column='completion_ratio',
        training_job_name='RidgeRegr'+'_'+suffix,
        model_type='Ridge',
    )    

    end = time()
    print("Total time", end-start,"s")
    update_results(results, result_line)

    # Random Forest regressor
    result_line = global_pipeline_train(
        all_features=selected_columns,
        numerical_features=selected_columns,
        params_grid=[
            {
                'clf__max_depth':[ 16],
            },
        ],
        score='neg_root_mean_squared_error',
        score_cls = mean_squared_error,
        X_train_val=X_train_val,
        X_test=X_test,
        target_column='completion_ratio',
        training_job_name='RandomForestRegr'+'_'+suffix,
        model_type='RandomForestRegressor',
    )    

    end = time()
    print("Total time", end-start,"s")
    update_results(results, result_line)
    # KNN-regressor
    # result_line = global_pipeline_train(
    #     all_features=selected_columns,
    #     numerical_features=selected_columns,
    #     params_grid=[
    #         {
    #             'clf__n_neighbors':[5,],
    #         },
    #     ],
    #     score='neg_root_mean_squared_error',
    #     score_cls = mean_squared_error,
    #     X_train_val=X_train_val,
    #     X_test=X_test,
    #     target_column='completion_ratio',
    #     training_job_name='KNeighborsRegr'+'_'+suffix,
    #     model_type='KNeighborsRegressor',
    # )    

    #end = time()
    #print("Total time", end-start,"s")
    #update_results(results, result_line)
    return pd.DataFrame(results).round(5)

In [62]:

df = pd.read_csv('../data/completion_rate.csv')
selected_columns = list(df.columns)
selected_columns.pop(selected_columns.index('form_id'))
selected_columns.pop(selected_columns.index('views'))
selected_columns.pop(selected_columns.index('submissions'))

df = df.sample(frac=0.7)
print(len(df))
X_train_val, X_train, X_validation, X_test = split_dataset(df)
print(len(X_train_val),len(X_train),len(X_validation),len(X_test))

my_current_train(restart=False, suffix='bigger_sample')

721899
577519 462015 115504 144380
Total time 6.457197189331055 s
Total time 9.049171209335327 s
Total time 1295.1183230876923 s


,name,prefix,model_class,trained_score,scorer_func,params,test_score
0,LinearRegr,clf,LinearRegression(),-0.21631,mean_squared_error,{},0.21623
1,RidgeRegr,clf,Ridge(),-0.21631,mean_squared_error,{'clf__alpha': 0.01},0.21622
2,RandomForestRegr,clf,RandomForestRegressor(),-0.20925,mean_squared_error,{'clf__max_depth': 16},0.20890
3,LinearRegr_bigger_sample,clf,LinearRegression(),-0.21627,mean_squared_error,{},0.21680
4,RidgeRegr_bigger_sample,clf,Ridge(),-0.21627,mean_squared_error,{'clf__alpha': 0.5},0.21680
5,RandomForestRegr_bigger_sample,clf,RandomForestRegressor(),-0.20891,mean_squared_error,{'clf__max_depth': 16},0.20900


### Using less features expecting a drop in performance

In [63]:

df = pd.read_csv('../data/completion_rate.csv')
selected_columns = list(df.columns)
selected_columns.pop(selected_columns.index('form_id'))
selected_columns.pop(selected_columns.index('views'))
selected_columns.pop(selected_columns.index('submissions'))
selected_columns = ['feat_29','feat_05']
"""
['feat_01', 'feat_02', 'feat_03', 'feat_04', 'feat_05', 'feat_06', 'feat_07', 'feat_08', 'feat_09', 'feat_10', 'feat_11', 'feat_12', 'feat_13', 'feat_14', 'feat_15', 'feat_16', 'feat_17', 'feat_18', 'feat_19', 'feat_20', 'feat_21', 'feat_22', 'feat_23', 'feat_24', 'feat_25', 'feat_26', 'feat_27', 'feat_28', 'feat_29', 'feat_30', 'feat_31', 'feat_32', 'feat_33', 'feat_34', 'feat_35', 'feat_36', 'feat_37', 'feat_38', 'feat_39', 'feat_40', 'feat_41', 'feat_42', 'feat_43', 'feat_44', 'feat_45', 'feat_46', 'feat_47']
"""

df = df.sample(frac=0.3)
print(len(df))
X_train_val, X_train, X_validation, X_test = split_dataset(df)
print(len(X_train_val),len(X_train),len(X_validation),len(X_test))

my_current_train(restart=False, suffix='_less_features')

309385
247508 198006 49502 61877
Total time 0.24765515327453613 s
Total time 0.4654717445373535 s
Total time 12.706403732299805 s


,name,prefix,model_class,trained_score,scorer_func,params,test_score
0,LinearRegr,clf,LinearRegression(),-0.21631,mean_squared_error,{},0.21623
1,RidgeRegr,clf,Ridge(),-0.21631,mean_squared_error,{'clf__alpha': 0.01},0.21622
2,RandomForestRegr,clf,RandomForestRegressor(),-0.20925,mean_squared_error,{'clf__max_depth': 16},0.20890
3,LinearRegr_bigger_sample,clf,LinearRegression(),-0.21627,mean_squared_error,{},0.21680
4,RidgeRegr_bigger_sample,clf,Ridge(),-0.21627,mean_squared_error,{'clf__alpha': 0.5},0.21680
5,RandomForestRegr_bigger_sample,clf,RandomForestRegressor(),-0.20891,mean_squared_error,{'clf__max_depth': 16},0.20900
6,LinearRegr__less_features,clf,LinearRegression(),-0.22349,mean_squared_error,{},0.22344
7,RidgeRegr__less_features,clf,Ridge(),-0.22349,mean_squared_error,{'clf__alpha': 0.5},0.22344
8,RandomForestRegr__less_features,clf,RandomForestRegressor(),-0.22351,mean_squared_error,{'clf__max_depth': 16},0.22347


In [64]:
df = pd.read_csv('../data/completion_rate.csv')
selected_columns = list(df.columns)
selected_columns.pop(selected_columns.index('form_id'))
selected_columns.pop(selected_columns.index('views'))
selected_columns.pop(selected_columns.index('submissions'))
selected_columns = ['feat_09']
"""
['feat_01', 'feat_02', 'feat_03', 'feat_04', 'feat_05', 'feat_06', 'feat_07', 'feat_08', 'feat_09', 'feat_10', 'feat_11', 'feat_12', 'feat_13', 'feat_14', 'feat_15', 'feat_16', 'feat_17', 'feat_18', 'feat_19', 'feat_20', 'feat_21', 'feat_22', 'feat_23', 'feat_24', 'feat_25', 'feat_26', 'feat_27', 'feat_28', 'feat_29', 'feat_30', 'feat_31', 'feat_32', 'feat_33', 'feat_34', 'feat_35', 'feat_36', 'feat_37', 'feat_38', 'feat_39', 'feat_40', 'feat_41', 'feat_42', 'feat_43', 'feat_44', 'feat_45', 'feat_46', 'feat_47']
"""

df = df.sample(frac=0.3)
print(len(df))
X_train_val, X_train, X_validation, X_test = split_dataset(df)
print(len(X_train_val),len(X_train),len(X_validation),len(X_test))

my_current_train(restart=False, suffix='_less_features2')

309385
247508 198006 49502 61877
Total time 0.12825560569763184 s
Total time 0.30943870544433594 s
Total time 6.952027082443237 s


,name,prefix,model_class,trained_score,scorer_func,params,test_score
0,LinearRegr,clf,LinearRegression(),-0.21631,mean_squared_error,{},0.21623
1,RidgeRegr,clf,Ridge(),-0.21631,mean_squared_error,{'clf__alpha': 0.01},0.21622
2,RandomForestRegr,clf,RandomForestRegressor(),-0.20925,mean_squared_error,{'clf__max_depth': 16},0.20890
3,LinearRegr_bigger_sample,clf,LinearRegression(),-0.21627,mean_squared_error,{},0.21680
4,RidgeRegr_bigger_sample,clf,Ridge(),-0.21627,mean_squared_error,{'clf__alpha': 0.5},0.21680
5,RandomForestRegr_bigger_sample,clf,RandomForestRegressor(),-0.20891,mean_squared_error,{'clf__max_depth': 16},0.20900
6,LinearRegr__less_features,clf,LinearRegression(),-0.22349,mean_squared_error,{},0.22344
7,RidgeRegr__less_features,clf,Ridge(),-0.22349,mean_squared_error,{'clf__alpha': 0.5},0.22344
8,RandomForestRegr__less_features,clf,RandomForestRegressor(),-0.22351,mean_squared_error,{'clf__max_depth': 16},0.22347
9,LinearRegr__less_features2,clf,LinearRegression(),-0.22129,mean_squared_error,{},0.22061


In [65]:
df = pd.read_csv('../data/completion_rate.csv')
selected_columns = list(df.columns)
selected_columns.pop(selected_columns.index('form_id'))
selected_columns.pop(selected_columns.index('views'))
selected_columns.pop(selected_columns.index('submissions'))
selected_columns.pop(selected_columns.index('feat_05'))
selected_columns.pop(selected_columns.index('feat_07'))
selected_columns.pop(selected_columns.index('feat_33'))
selected_columns.pop(selected_columns.index('feat_45'))
selected_columns.pop(selected_columns.index('feat_23'))

"""
['feat_01', 'feat_02', 'feat_03', 'feat_04', 'feat_05', 'feat_06', 'feat_07', 'feat_08', 'feat_09', 'feat_10', 'feat_11', 'feat_12', 'feat_13', 'feat_14', 'feat_15', 'feat_16', 'feat_17', 'feat_18', 'feat_19', 'feat_20', 'feat_21', 'feat_22', 'feat_23', 'feat_24', 'feat_25', 'feat_26', 'feat_27', 'feat_28', 'feat_29', 'feat_30', 'feat_31', 'feat_32', 'feat_33', 'feat_34', 'feat_35', 'feat_36', 'feat_37', 'feat_38', 'feat_39', 'feat_40', 'feat_41', 'feat_42', 'feat_43', 'feat_44', 'feat_45', 'feat_46', 'feat_47']
"""

df = df.sample(frac=0.3)
print(len(df))
X_train_val, X_train, X_validation, X_test = split_dataset(df)
print(len(X_train_val),len(X_train),len(X_validation),len(X_test))

my_current_train(restart=False, suffix='_remove_correlated')

309385
247508 198006 49502 61877
Total time 2.573469877243042 s
Total time 3.7292325496673584 s
Total time 408.7744674682617 s


,name,prefix,model_class,trained_score,scorer_func,params,test_score
0,LinearRegr,clf,LinearRegression(),-0.21631,mean_squared_error,{},0.21623
1,RidgeRegr,clf,Ridge(),-0.21631,mean_squared_error,{'clf__alpha': 0.01},0.21622
2,RandomForestRegr,clf,RandomForestRegressor(),-0.20925,mean_squared_error,{'clf__max_depth': 16},0.20890
3,LinearRegr_bigger_sample,clf,LinearRegression(),-0.21627,mean_squared_error,{},0.21680
4,RidgeRegr_bigger_sample,clf,Ridge(),-0.21627,mean_squared_error,{'clf__alpha': 0.5},0.21680
5,RandomForestRegr_bigger_sample,clf,RandomForestRegressor(),-0.20891,mean_squared_error,{'clf__max_depth': 16},0.20900
6,LinearRegr__less_features,clf,LinearRegression(),-0.22349,mean_squared_error,{},0.22344
7,RidgeRegr__less_features,clf,Ridge(),-0.22349,mean_squared_error,{'clf__alpha': 0.5},0.22344
8,RandomForestRegr__less_features,clf,RandomForestRegressor(),-0.22351,mean_squared_error,{'clf__max_depth': 16},0.22347
9,LinearRegr__less_features2,clf,LinearRegression(),-0.22129,mean_squared_error,{},0.22061
